<a href="https://colab.research.google.com/github/hardikkamboj/Google-Earth-Engine-Notebooks/blob/main/EEMONT_tutorials/001-Clouds-and-Shadows-Masking-Sentinel-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clouds/Shadows Masking on Sentinel-2 Surface Reflectance Product
_Tutorial created by **David Montero Loaiza**_: [GitHub](https://github.com/davemlz) | [Twitter](https://twitter.com/dmlmont)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/docs/tutorials](https://github.com/davemlz/eemont/tree/master/docs/tutorials)

## Let's start!

If required, please uncomment:

In [1]:
!pip install eemont
!pip install geemap

     |████████████████████████████████| 116 kB 5.2 MB/s 
     |████████████████████████████████| 165 kB 47.8 MB/s 
  Created wheel for eemont: filename=eemont-0.3.0-py3-none-any.whl size=116909 sha256=eb91204e6ff9fcf3d243057475e7a71222c754babeb10601cee1c37158360978
  Stored in directory: /root/.cache/pip/wheels/dc/d7/f7/c1f7cc748e03d7184321c7d83785c4076c74c96f878c9b6a36
  Created wheel for ee-extra: filename=ee_extra-0.0.9-py3-none-any.whl size=174156 sha256=21967a8b7f4e9df92d7f72533253723c85eb37bebc0da961a10ec219e0b66f17
  Stored in directory: /root/.cache/pip/wheels/c7/2a/f2/072818a085226599bf844cb3c9b3e40758d182f55040a8afba
Successfully built eemont ee-extra
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 130 kB 58.7 MB/s 
     |████████████████████████████████| 219 kB 55.0 MB/s 
     |████████████████████████████████| 98 kB 7.5 MB/s 
     |████████████████████████████████| 1.2 M

Import the required packges.

In [1]:
import ee, eemont, geemap

Authenticate and Initialize Earth Engine and geemap.

In [2]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PSo245Af9k6cZuNlWZak5FW7FDz8lvIx1kP7nBkd9oA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgbPFhryJrdsIBuv_v0chAsZ7ZucDYQGvFcG3CiJPMybdcv4sTW9fM

Successfully saved authorization token.


Example point of interest to filter the image collection.

In [3]:
point = ee.Geometry.Point([-75.92, 2.86])

Get and filter the Sentinel-2 Surface Reflectance image collection and filter it by region and time.

In [4]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .filterDate('2020-01-01','2021-01-01'))

## Clouds/Shadows Masking
Multiple options are included in this feature, let's take a look at them:

### 1. Quality Assessment masking
Here, the Quality Assessment band is used for masking.

In [5]:
S2a = S2.maskClouds(method = 'qa')

### 2. Quality Assessment masking (without masking cirrus clouds)
By default, cirrus clouds are masked, but if required, the *maskCirrus* parameter can be set to *False*.

In [6]:
S2b = S2.maskClouds(method = 'qa', maskCirrus = False)

### 3. Cloud Probability masking
Cloud Probability masking is the default method to mask clouds and shadows in Sentinel-2 (method = 'cloud_prob'). By default, the Cloud Probability threshold is 60% (prob = 60).

In [7]:
S2c = S2.maskClouds()

### 4. Cloud Probability masking with a different threshold value
The threshold value for Cloud Probability can be changed. Values in the range [0.0, 100.0] are accepted.

In [8]:
S2d = S2.maskClouds(prob = 75)

### 5. Cloud Probability masking with a different buffer value
By default, clouds and shadows are dilated by a 250 m buffer to avoid border effects. This value can be changed by modifying the *buffer* parameter.

In [9]:
S2e = S2.maskClouds(buffer = 500)

### 6. Cloud Probability masking with a different NIR threshold
By default, NIR values under 0.15 are considered potential cloud shadows. This value can be changed by modifying the *dark* parameter.

In [10]:
S2f = S2.maskClouds(dark = 0.1)

### 7. Cloud Probability masking with a different shadow search range
By default, cloud shadows are searched whithin a 1000 m distance from cloud edges in the shadow direction. This value can be changed by modifying the *cloudDist* parameter.

In [11]:
S2g = S2.maskClouds(cloudDist = 2000)

### 8. Cloud Probability masking using the Cloud Displacement Index (CDI)
CDI is an index used to avoid masking bright surfaces as clouds. By default, CDI is not used, but if required, the CDI value can be changed by modifying the *cdi* parameter. CDI values go from -1 to 1. The most used value for cloud masking is -0.5.

In [12]:
S2h = S2.maskClouds(cdi = -0.5)

### 9. Cloud Probability masking (without masking shadows)
By default, shadows are masked as well as clouds, but if required, the *maskShadows* parameter can be set to *False*. This option is available for 'qa' and 'cloud_prob' methods.

In [13]:
S2i = S2.maskClouds(maskShadows = False)

### 10. Cloud Probability masking of a scaled image (pixel values in [0, 1])
By default, clouds and shadows are masked on unscaled image collections, but if the collection is scaled, the *scaledImage* MUST be set to *True*.

In [14]:
S2j = S2.scale().maskClouds(scaledImage = True)

## Visualization

Set the visualization parameters.

In [15]:
rgbUnscaled = {'min':0, 'max':3000, 'bands':['B4','B3','B2']}
rgbScaled = {'min':0, 'max':0.3, 'bands':['B4','B3','B2']}

Use geemap to display results.

In [16]:
Map.centerObject(point,10)
Map.addLayer(S2.first(),rgbUnscaled,'No Clouds/Shadows masking')
Map.addLayer(S2a.first(),rgbUnscaled,'QA masking')
Map.addLayer(S2b.first(),rgbUnscaled,'QA masking with no cirrus masking')
Map.addLayer(S2c.first(),rgbUnscaled,'60% Cloud Probability masking')
Map.addLayer(S2d.first(),rgbUnscaled,'75% Cloud Probability masking')
Map.addLayer(S2e.first(),rgbUnscaled,'60% Cloud Probability masking with a 500 m dilation')
Map.addLayer(S2f.first(),rgbUnscaled,'60% Cloud Probability masking with a 0.1 NIR threshold')
Map.addLayer(S2g.first(),rgbUnscaled,'60% Cloud Probability masking with a 2000 m cloud shadow search range')
Map.addLayer(S2h.first(),rgbUnscaled,'60% Cloud Probability masking with a CDI of -0.5')
Map.addLayer(S2i.first(),rgbUnscaled,'60% Cloud Probability masking with no shadow masking')
Map.addLayer(S2j.first(),rgbScaled,'60% Cloud Probability masking of a scaled image')
Map

Map(center=[2.86, -75.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…